In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.nn.init import kaiming_uniform_

In [2]:
import numpy as np
import gym
from tqdm import tqdm
import random as rand
from itertools import count

In [3]:
class OU_noise():
    def __init__(self,mu,sigma=0.3,theta=0.15,dt=1e-5,x0=None):
        self.theta = theta
        self.mu    = mu
        self.sigma = sigma
        self.dt    = dt
        self.x0    = x0
        self.reset()
        
    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev)* \
                self.dt +self.sigma * np.sqrt(self.dt)*np.random.normal(size = self.mu.shape)
        self.x_prev = x
        return x
    
    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

In [ ]:
class linearApproximator_FCGSAP(nn.Module):
    def __init__(self,state_shape,outputs,hidden_dims=(32,32)):
        super(linearApproximator_FCGSAP, self).__init__()
        self.input_size = state_shape
        self.out = outputs
        self.device = torch.device("cuda" if torch.cuda.is_available()\
                                   else "cpu")
        
        self.fc1  = nn.Linear(self.input_size,hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            hidden_layer = nn.Linear(\
                                hidden_dims[i], hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        
        self.output_layer_log  = nn.Linear(hidden_dims[-1],self.out)
        self.output_layer_mean = nn.Linear(hidden_dims[-1],self.out)
        self.to(self.device)
        
    def forward(self, state_shape):
        if not isinstance(state_shape, torch.Tensor):
            state_shape = torch.tensor(state_shape, dtype=torch.float32)
        state_shape = state_shape.to(self.device)
            
        x = self.fc1(state_shape)
        x = F.relu(x)
        
        for hidden_layer in self.hidden_layers:
            x = F.relu(hidden_layer(x))
        
        logits = self.output_layer(x)#logits, preferences of actions
        return logits
        
    def full_pass(self, state):
        logits = self.forward(state).cpu()
        distribution = Categorical(logits=logits)
        action = distribution.sample()#sample action
        log_prob_action = distribution.log_prob(action).unsqueeze(-1)#gets prob of sampled action
        entropy = distribution.entropy().unsqueeze(-1)
        return action.item(), log_prob_action, entropy, logits

In [1]:
import numpy as np
-np.prod(4)

-4

In [13]:
import gym
gym.make('BipedalWalker-v3')

AttributeError: module 'gym.envs.box2d' has no attribute 'BipedalWalker'

In [10]:
%pip install ribs[all] gym~=0.17.0 Box2D~=2.3.10 tqdm

  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654651 sha256=18a25c25845a14f8a73a997b9dc286cb36c4bcdf46886e229161df92ed49f02b
  Stored in directory: c:\users\scowt\appdata\local\pip\cache\wheels\d1\81\4b\dd9c029691022cb957398d1f015e66b75e37637dda61abdf58
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.18.0
    Uninstalling gym-0.18.0:
      Successfully uninstalled gym-0.18.0
Note: you may need to restart the kernel to use updated packages.


distutils: c:\users\scowt\appdata\local\programs\python\python37\Include\UNKNOWN
sysconfig: c:\users\scowt\appdata\local\programs\python\python37\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\scowt\appdata\local\programs\python\python37\Include\UNKNOWN
sysconfig: c:\users\scowt\appdata\local\programs\python\python37\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\scowt\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.
